# Simulated Annealing

#### By Jorge Almonacid


Simulated Annealing is an iterative algorithm that explores the solution space by occasionally accepting worse solutions with a probability that decreases over time, enabling the algorithm to escape local optima and eventually settle into a global optimum (or a good approximation).

In [ ]:
# Library imports

import numpy as np
import pandas as pd
import math

In [ ]:
# Function Definition

def neighborhood(lista1: list) -> list:
    """
    This function creates a list from the adjacent neighbour space to the list passed.
    E.g. lista1 = [0,0] -> lista =  [0,1] || [1,0]
    
    Args:
        lista1 (list): List you want to obtain the neighbour from
        
    Returns: 
        lista (list): Neighbour list   
    """
    lista = lista1.copy()
    i = np.random.randint(0, len(lista))
    lista[i] = 1 - lista[i]
    return lista

def list_to_str(lista:list):
    """ 
    This function is for converting a list into a string format, so the dataframe can save it
    as a single value. 
    E.g. lista = [1,0,1,1] -> t = 1011
    
    Args: 
        Lista (list): list to transform into string
        
    Return: 
        t (str): String associated to the list
    """
    t = ""
    for elem in lista:
        l = t
        s = str(elem)
        t = l+s
    return t

def evaluation(weighs:list) -> float:
    def compare(list1: list) -> list:
        """ 
        This function evaluates a list associated to its positional weighs. Both lists must contain numbers.
        E.g. weighs = [1,1,2,0], list1 = [0,1,1,0] -> [1*0 + 1*1 + 2*1 + 0*0] = 3
        
        Args:
            list1 (list): List you want to calculate the weighted value from
            weighs (list): Weighs associated to each list position
            
        Returns: 
            compare (function): Function with the associated weighs
            (float): Total added value from the list   
        """
        return np.dot(list1,weighs)
    return compare
    
def valid(max_coste:float, coste:list, lista:list) -> bool:
    """ 
    This function checks if the cost associated to a list exceeds or not the maximum cost.
    E.g. max_cost = 10, cost = [5,6,1], lista = [1,1,0]: 10 < (5*1 + 6*1)=11 -> False   
    
    Args: 
        max_coste (float): Max cost you can't surpass
        coste (list): Weighs associated to the list
        lista (list):  List you want to check if it's valid
        
    Returns:
        (bool): True if the value is under the maximum cost, False otherwise
    """
    coste_total = np.dot(coste, lista)
    if max_coste < coste_total:
        return False
    return True

def cooling(temp: float, cool_rate: float) -> float:
    """ 
    Function to reduce the temperature by some certain cooling rate.
    E.g. temp = 10, cool_rate = 0.9 -> 10*0.9 = 9
    
    Args:
        temp (float): Temperature value you want to decrease
        cool_rate (float): Temperature decrease rate
        
    Returns:
        (float): New temperature after cooling down
    """
    return cool_rate * temp


def simulated_annealing(lista1: list, temp: float, pesos: list, max_cost: float, coste: list, cool_rate: float, df: pd.DataFrame) -> list:
    """ 
    This function solves a simulated annealing problem given the starting point list, initial temperature, weighs associated to the problem,
    maximum cost we want to not surpass, costs associated to the problem , cooling rate associated to the temperature and a dataframe
    to save the outputs later.
    
    Args:
        lista1 (list): Starting point for the problem. In our case a 0 filled n length matrix
        temp (float): Initial temperature 
        pesos (list): Weighs associated to the problem
        max_cost (float): Cost that we must not surpass
        coste (list): Costs associated to the problem
        cool_rate (float): Temperature decrease rate
        df (pd.DataFrame): Dataframe we will modify to the save it later in the code
        
    Returns:
        best (list): List with the maximum associated value we have found that does not surpass the maximum cost.
        best_eval (float): Value associated to the best list
        (float): Cost associated to the best list.
        df (pd.DataFrame): Dataframe with the outputs from the problem each 100 iterations ("Iteración","Lista", "Valor", "Peso", "Temperatura", "Energy")
    """
    # Firstly we initialize the solution and best lists, to keep track of them and define a function with the associated weighs for the problem
    solution = lista1.copy()
    eval = evaluation(pesos)
    best = lista1.copy()
    i=0
    # While loop for evaluating until the temperature drops below 1
    while temp>1:
        # Evaluate the best list
        best_eval = (eval(best))
        # Generate random point between 0 and 1 = (exp(-(ev1-ev2)/temp)) to compare with the energy, when ev1 <= ev2
        rn = np.random.uniform(0,1)
        # Generate a neighbour to the current solution list
        lista2 = neighborhood(solution)
        # Reduce the temperature
        temp = cooling(temp, cool_rate)
        # Evaluate both lists, current solution and possible solution
        ev1 = eval(solution)    
        ev2 = eval(lista2)
        
        # Check if the new list is exceeding the maximum cost to set its value to 0
        if np.dot(coste, lista2) > max_cost:
            ev2 = 0
        
        # Define the Δ between both evaluations and calculate the energy
        dif = ev2 - ev1
        energy = math.exp(dif/temp)
        
        # If the random value is smaller than the energy, or if ev2 > ev1, we set the solution as the new list
        if (dif > 0) or (energy > rn):
            solution = lista2.copy()
        
        # If the new list has a greater evaluation we also modify the best solution
        if ev2 > best_eval:
            best = solution.copy()
        
        # Add 1 for dataframe iteration count
        i+=1
        
        # DataFrame column addition every 100 iterations
        ev = eval(solution)
        
        if np.dot(coste, solution) > max_cost:
            ev = 0
        if i% 100 == 0:
            df2 = pd.DataFrame(np.array([[i,list_to_str(solution),ev,np.dot(coste, solution),temp, energy]]), columns=["Iteración","Lista", "Valor", "Peso", "Temperatura", "Energy"])       
            df = pd.concat([df,df2])
    
    return best, best_eval, np.dot(coste, best), df




In [ ]:
# Define the problem
# Weighs and costs associated to the problem
pesos = [12,10,20,15,18]
coste = [4,6,5,3,7]
n = len(coste)

sols = [0]*n

# Maximum cost, temperature and cooling rate
max_cost = 15
temp = 100000
cool_rate = 0.999

# DataFrame initialization
df = pd.DataFrame(np.array([[0, list_to_str(sols), evaluation(pesos)(sols), 0, temp, 10]]), columns= ["Iteración","Lista", "Valor", "Peso", "Temperatura","Energy"])

# Problem solving
sol = simulated_annealing(sols, temp, pesos, max_cost, coste, cool_rate,df)
print(sol[:3])

([0, 0, 1, 1, 1], np.int64(53), np.int64(15))


In [ ]:
# Save solutions into a csv
df3 = sol[3]
df3.to_csv("simmulated_annealing_output.csv")
print(df3)

    Index  Lista Valor Peso         Temperatura                  Energy
0       0  00000     0    0              100000                      10
0     100  11101     0   22   90479.21471137082       0.999447539555065
0     200  00011    33   10   81864.88294786352      0.9998778549617088
0     300  10010    27    7   74070.70321560993      0.9998650029692502
0     400  11110     0   18   67018.59060067407      0.9993287698946023
..    ...    ...   ...  ...                 ...                     ...
0   11100  00111    53   15  1.5028627093582578   4.832347698625809e-16
0   11200  00111    53   15  1.3597783776173842   1.782575621081914e-06
0   11300  00111    53   15  1.2303167978832295  1.9557643650795706e-19
0   11400  00111    53   15    1.11318097718683   1.574834558865885e-08
0   11500  00111    53   15  1.0071974064750093   3.405137296758386e-07

[116 rows x 6 columns]
